In [1]:
# Sentences we'll respond with if the user greeted us
GREETING_KEYWORDS = ("hello", "hi", "greetings", "sup", "what's up",)

GREETING_RESPONSES = ["'sup bro", "hey", "*nods*", "hey you get my snap?"]

def check_for_greeting(sentence):
    """If any of the words in the user's input was a greeting, return a greeting response"""
    for word in sentence.words:
        if word.lower() in GREETING_KEYWORDS:
            return random.choice(GREETING_RESPONSES)


In [7]:
def response(sentence):
    """Parse the user's inbound sentence and find candidate terms that make up a bet-fit response"""
    cleaned = preprocess_text(sentence)
    parsed = TextBlob(cleaned)
    
    # Loot through all the sentences, if more than one. This will help extract the most relevant 
    # response text even across multiplt sentences (for example if there was no obvius direct noun
    # in one sentence
    pronoun, noun, adjective, verb = find_candidate_parts_of_speech(parsed)
    
    # If we said something about the bot and used some kind of direct noun, construct the 
    # sentence around that, discarding the other candidates
    resp = check_for_comment_about_bot(pronoun, noun, adjective)
    
    # If we just greeted the bot, we'll use a return greeting
    if not resp:
        resp = check_for_greeting(parsed)
    
    if not resp:
        # if we didn't override the final sentence, try to construct a new one:
        if not pronoun:
            resp = random.choice(NONE_RESPONSES)
        elif pronoun == 'I' and not verb:
            resp = random.choice(COMMENTS_ABOUT_SELF)
        else:
            resp = construct_response(pronoun, noun, verb)
        
    # If we got through all that with nothing, use a random response
    if not resp:
        resp = random.choice(NONE_RESPONSES)
        
    logger.info("Returning phrase '%s'", resp)
    # Check that we're not going to say anything obviously offensive
    fliter_response(resp)
    
    def find_candidate_parts_of_speech(parsed):
        """Given a parsed input, find the best pronoun, direct noun, adjective, and verb to match their input
        Returns a tuple of pronoun, noun, adjective, verb any of which may be None if there was no good match"""
        pronoun = None
        noun = None
        adjective = None
        verb = None
        for sent in parsed.sentences:
            pronoun = find_pronoun(sent)
            noun = find_noun(sent)
            adjective = find_adjective(sent)
            verb = find_verb(sent)
    
        logger.info("Pronoun=%s, noun=%s, adjective=%s, verb=%s", pronoun, noun, adjective, verb)
        return pronoun, noun, adjective, verb